In [116]:
# Import modules
import sys, subprocess, shutil
import os, time, itertools
import re, csv

# from pydsstools.heclib.dss import HecDss
import numpy as np
import pandas as pd

In [117]:
#Logic to determine HMS Project Name

# Get HMS Project Name from hms_project_directory
def get_hms_project_name(hms_project_directory):
    """
    Given an HMS project directory, return the HMS project name.
    The project name is determined based on the .hms file in the directory.
    If more than one .hms file exists or none exist, an appropriate error message is returned.

    Args:
    - hms_project_directory (str): Path to the HMS project directory

    Returns:
    - str: HMS project name or error message
    """
    # List all files in the directory
    all_files = os.listdir(hms_project_directory)

    # Filter for files with .hms extension
    hms_files = [f for f in all_files if f.endswith('.hms')]

    # Check if there's exactly one .hms file
    if len(hms_files) == 1:
        # Extract the project name by stripping the .hms extension
        return hms_files[0].replace('.hms', '')
    elif len(hms_files) > 1:
        return "Error: More than one .hms file exists in the directory!"
    else:
        return "Error: No .hms file found in the directory!"

In [118]:
# Define Project Paths and Working Paths

# Define project paths
hms_project_directory = r"D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy"
hms_basin_file_path = r"D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin"

# Define HMS run name
hms_run_name = 'edc_run_v1'

# ---- Define Working Paths ---- 

hms_dss_suffix = '_test_run'
print(f'DSS Suffix: {hms_dss_suffix}')

hms_project_name = get_hms_project_name(hms_project_directory)
hms_project_run_file = f"{hms_project_name}.run"

# Print Project Name or Error
print("HMS Project Name: ", hms_project_name)

hms_project_run_file_path = os.path.join(hms_project_directory, hms_project_run_file)
hms_project_run_file_backup_path = hms_project_run_file_path + ".bak"
hms_basin_file_backup_path = hms_basin_file_path + ".bak"

# ---- Additional Settings ----

# Define HEC-HMS Folder Path
hms_executable_path = r"C:\Program Files\HEC\HEC-HMS\4.12"
print(f'HEC-HMS Executable Path: {hms_executable_path}')

# Define paths for HMScompute.py and HMScompute.bat which are used to run HMS through Jython
hms_compute_py_path = r"D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\Python\HMScompute.py"
hms_compute_bat_path = r"D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\Python\HMScompute.bat"

''' If jython heap size is too small, the HMS will freeze at 100% CPU and never finish'''
jython_initial_heap_size = "256m"       #initial heap size for java virtual machine
print("Jython Initial Heap Size: " + jython_initial_heap_size)
jython_maximum_heap_size = "4096m"      #maximum heap size for java virtual machine
print("Jython Maximum Heap Size: " + jython_maximum_heap_size)

DSS Suffix: _test_run
HMS Project Name:  hms_edc_amacan
HEC-HMS Executable Path: C:\Program Files\HEC\HEC-HMS\4.12
Jython Initial Heap Size: 256m
Jython Maximum Heap Size: 4096m


In [119]:
# Check if Jython exists and Build HMScompute.bat and HMScompute.py

# Check for Jython 2.7.4 at C:\jython2.7.4

jython_path = r"C:\jython2.7.4"
jython_jar_path = os.path.join(jython_path, "jython.jar")

def jython_exists(path):
    exists = os.path.exists(path)
    return exists

def print_installation_instructions():
    print("Jython is not found at the provided path.")
    print("Please install the necessary software:")
    print("1. Java: https://www.java.com/en/download/")
    print("2. Java SE Development Kit (check for the latest version): https://www.oracle.com/java/technologies/javase-jdk-downloads.html")
    print("3. Jython: https://www.jython.org/download.html")

if jython_exists(jython_jar_path):
    print(f"Jython Exists at {jython_jar_path}")
else:
    print_installation_instructions()

# ---- Build settings ----

# Build HMScompute.bat and HMScompute.py for Jython 2.7.3


# Define the path and content for HMScompute.py
hms_compute_py_content = '''
from hms.model import Project
from hms import Hms

import glob
import sys

hmspth = sys.argv[1]
runName = sys.argv[2]
# print 'running' + str(hmspth)

myProject = Project.open(hmspth)
myProject.computeRun(runName)
myProject.close()

Hms.shutdownEngine()
'''

# Define the path and content for HMScompute.bat
hms_compute_bat_content = f'''@echo off
setlocal

set "PATH=C:\\Program Files\\HEC\\HEC-HMS\\4.12\\bin\\gdal;%PATH%"
set "GDAL_DATA=C:\\Program Files\\HEC\\HEC-HMS\\4.12\\bin\\gdal\\gdalplugins"
set "GDAL_DATA=C:\\Program Files\\HEC\\HEC-HMS\\4.12\\bin\\gdal\\gdal-data"
set "PROJ_LIB=C:\\Program Files\\HEC\\HEC-HMS\\4.12\\bin\\gdal\\projlib"

set "CLASSPATH=C:\\Program Files\\HEC\\HEC-HMS\\4.12\\hms.jar;C:\\Program Files\\HEC\\HEC-HMS\\4.12\\lib\\*"

"C:\\jython2.7.4\\bin\\jython.exe" ^
  -J-Xms{jython_initial_heap_size} -J-Xmx{jython_maximum_heap_size} ^
  -Djava.library.path="C:\\Program Files\\HEC\\HEC-HMS\\4.12\\bin;C:\\Program Files\\HEC\\HEC-HMS\\4.12\\bin\\gdal" ^
  "{hms_compute_py_path}" "%~1" "%~2"
'''


# Write the content for HMScompute.py
with open(hms_compute_py_path, 'w') as py_file:
    py_file.write(hms_compute_py_content)
print(f"Wrote content to {hms_compute_py_path}")


# Write the content for HMScompute.bat
with open(hms_compute_bat_path, 'w') as bat_file:
    bat_file.write(hms_compute_bat_content)
print(f"Wrote content to {hms_compute_bat_path}")

Jython Exists at C:\jython2.7.4\jython.jar
Wrote content to D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\Python\HMScompute.py
Wrote content to D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\Python\HMScompute.bat


In [120]:
# Calculate and Print necessary paths and file names

# Calculated Paths and File Names (.run, .basin, backup paths)
hms_project_run_file = f"{hms_project_name}.run"
hms_project_run_file_path = os.path.join(hms_project_directory, hms_project_run_file)
hms_basin_file_path = os.path.join(hms_project_directory, hms_basin_file_path)
hms_project_run_file_backup_path = hms_project_run_file_path + ".bak"
hms_basin_file_backup_path = hms_basin_file_path + ".bak"

# Print statements

print("HMS Project Run File:", hms_project_run_file)
print("HMS Project Run File Path:", hms_project_run_file_path)
print("HMS Basin File:", hms_basin_file_path)
print("HMS Basin File Path:", hms_basin_file_path)
print("HMS Project Run File Backup Path:", hms_project_run_file_backup_path)
print("HMS Basin File Backup Path:", hms_basin_file_backup_path)

HMS Project Run File: hms_edc_amacan.run
HMS Project Run File Path: D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\hms_edc_amacan.run
HMS Basin File: D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin
HMS Basin File Path: D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin
HMS Project Run File Backup Path: D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\hms_edc_amacan.run.bak
HMS Basin File Backup Path: D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin.bak


In [121]:
# Create Lock File and Logic for Working File Backup and Restore
# Define the path for the lock file based on one of the .bak file paths
lock_file_directory = os.path.dirname(hms_project_run_file_backup_path)
lock_file_path = os.path.join(lock_file_directory, "HMSCommander.lock")

if os.path.exists(lock_file_path):
    user_confirmation = input("It appears the script did not run successfully and .bak files are still present "
                              "in the HMS folder. Please confirm that you want to restore the original "
                              ".basin, .run, and .grid files from backup. Type 'yes' or 'y' to confirm, "
                              "or any other key to cancel: ").lower()

    if user_confirmation in ['yes', 'y']:
        # Step 2: Restore files from backup
        for backup, original in [(hms_project_run_file_backup_path, hms_project_run_file_path),
                                 (hms_basin_file_backup_path, hms_basin_file_path),
                                 ]:
            try:
                if os.path.exists(backup):
                    shutil.copy(backup, original)
                    # Delete the .bak file
                    os.remove(backup)
                    print(f"Restored and deleted file {backup}")
                else:
                    print(f"Backup file {backup} does not exist. Skipping restore.")
            except Exception as e:
                print(f"Error restoring file from {backup} to {original}: {e}")

        # Step 3: Delete the lock file
        os.remove(lock_file_path)
    else:
        print("Restoration process cancelled by user. Please inspect the files manually.")

else:
    print("Lock file does not exist. Proceeding to backup files.")

# Step 4: Backup files
for original, backup in [(hms_project_run_file_path, hms_project_run_file_backup_path),
                         (hms_basin_file_path, hms_basin_file_backup_path)
                         ]:
    try:
        if os.path.exists(original):
            shutil.copy(original, backup)
            print(f"Backed up file from {original} to {backup}")
        else:
            print(f"Original file {original} does not exist. Skipping backup.")
    except Exception as e:
        print(f"Error backing up file from {original} to {backup}: {e}")

# Step 5: Create a new HMSCommander.lock file
with open(lock_file_path, 'w') as f:
    f.write("Lock file for HMS Commander operations.")

print("Lock file created.")

Restored and deleted file D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\hms_edc_amacan.run.bak
Restored and deleted file D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin.bak
Backed up file from D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\hms_edc_amacan.run to D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\hms_edc_amacan.run.bak
Backed up file from D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin to D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin.bak
Lock file created.


In [122]:
# Define compute functions

# Reads the content of the HMS Basin file
def read_basin_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# ---- Update Basin Parameters ----

# Update SMA Parameters
def update_sma_params(hms_basin_file_path):
    # Set a parameter multiplier
    multiplier = 1.1

    # Search - Replace Dictionary here:
    search_replace = {
        r"(Percent Impervious Area:\s*)([0-9]*\.?[0-9]+)":           multiplier,
        # r"(Initial Soil Storage Percent:\s*)([0-9]*\.?[0-9]+)":      multiplier,
        # r"(Initial Gw1 Storage Percent:\s*)([0-9]*\.?[0-9]+)":       multiplier,
        # r"(Initial Gw2 Storage Percent:\s*)([0-9]*\.?[0-9]+)":       multiplier,
        r"(Soil Maximum Infiltration:\s*)([0-9]*\.?[0-9]+)":         multiplier,
        r"(Soil Storage Capacity:\s*)([0-9]*\.?[0-9]+)":             multiplier,
        r"(Soil Tension Capacity:\s*)([0-9]*\.?[0-9]+)":             multiplier,
        r"(Soil Maximum Percolation:\s*)([0-9]*\.?[0-9]+)":          multiplier,
        r"(Groundwater 1 Storage Capacity:\s*)([0-9]*\.?[0-9]+)":    multiplier,
        r"(Groundwater 1 Routing Coefficient:\s*)([0-9]*\.?[0-9]+)": multiplier,
        r"(Groundwater 1 Maximum Percolation:\s*)([0-9]*\.?[0-9]+)": multiplier,
        r"(Groundwater 2 Storage Capacity:\s*)([0-9]*\.?[0-9]+)":    multiplier,
        r"(Groundwater 2 Routing Coefficient:\s*)([0-9]*\.?[0-9]+)": multiplier,
        r"(Groundwater 2 Maximum Percolation:\s*)([0-9]*\.?[0-9]+)": multiplier,
    }


    # Read and modify the file line by line
    with open(hms_basin_file_path, 'r') as f:
        text = f.read()

    # Apply each pattern
    for pattern, mult in search_replace.items():
        regex = re.compile(pattern, flags=re.IGNORECASE)

        def repl(m):
            label = m.group(1)
            old_val = float(m.group(2))
            new_val = old_val * mult
            # choose formatting as needed; here 3 decimal places
            return f"{label}{new_val:.3f}"

        text = regex.sub(repl, text)

    # Write the changes back
    with open(hms_basin_file_path, 'w') as f:
        f.write(text)

# Update Baseflow Recession Parameters
def update_baseflow_recession(hms_basin_file_path):
    # Baseflow Recesion Multiplier
    mult = 1.1

    search_replace = {
        r"(Recession Factor:\s*)([0-9]*\.?[0-9]+)":           mult,
        r"(Initial Baseflow:\s*)([0-9]*\.?[0-9]+)":           mult,
        r"(Threshold Flow to Peak Ratio:\s*)([0-9]*\.?[0-9]+)": mult,
    }

    # Read entire file
    with open(hms_basin_file_path, 'r') as f:
        text = f.read()

    # For each pattern, multiply the captured value
    for pattern, multiplier in search_replace.items():
        regex = re.compile(pattern, flags=re.IGNORECASE)

        def repl(m):
            label = m.group(1)            
            old_val = float(m.group(2))   
            new_val = old_val * multiplier
            return f"{label}{new_val:.3f}"  

        text = regex.sub(repl, text)

    # Write back
    with open(hms_basin_file_path, 'w') as f:
        f.write(text)

# Update Simpe Canopy Parameters
def update_simple_canopy(hms_basin_file_path):
    mult = 1.1

    search_replace = {
        # r"(Initial Canopy Storage Percent:\s*)([0-9]*\.?[0-9]+)": mult,
        r"(Canopy Storage Capacity:\s*)([0-9]*\.?[0-9]+)":        mult,
        # r"(Crop Coefficient:\s*)([0-9]*\.?[0-9]+)":               mult,
    }   
    
    # Read full text
    with open(hms_basin_file_path, 'r') as f:
        text = f.read()

    # Apply each pattern
    for pattern, multiplier in search_replace.items():
        regex = re.compile(pattern, flags=re.IGNORECASE)
        def repl(m):
            label = m.group(1)
            old = float(m.group(2))
            new = old * multiplier
            return f"{label}{new:.3f}"
        text = regex.sub(repl, text)

    # Write back
    with open(hms_basin_file_path, 'w') as f:
        f.write(text)  

# Update Simple Surface Storage Parameters
def update_simple_surface(hms_basin_file_path):
    mult = 1.1

    search_replace = {
        # r"(Initial Surface Storage Percent:\s*)([0-9]*\.?[0-9]+)": mult,
        r"(Surface Storage Capacity:\s*)([0-9]*\.?[0-9]+)":          mult,
        # r"(Surface Albedo:\s*)([0-9]*\.?[0-9]+)":                   mult,
    }

    # Read full text
    with open(hms_basin_file_path, 'r') as f:
        text = f.read()

    # Apply each pattern
    for pattern, multiplier in search_replace.items():
        regex = re.compile(pattern, flags=re.IGNORECASE)
        def repl(m):
            label = m.group(1)
            old = float(m.group(2))
            new = old * multiplier
            return f"{label}{new:.3f}"
        text = regex.sub(repl, text)

    # Write back
    with open(hms_basin_file_path, 'w') as f:
        f.write(text)

In [123]:
# Main Logic of script to Run HMS

update_simple_canopy(hms_basin_file_path=hms_basin_file_path)


hms_run_output_dss = f"{hms_run_name}.dss"
hms_run_output_dss_path = os.path.join(hms_project_directory, f"{hms_run_output_dss}")


print("Output DSS file does not exist. Preparing HMS Run.")

# Update the .run file witht the new DSS File name
with open(hms_project_run_file_path, 'r') as file:
       run_data = file.readlines()

run_found = False
old_dss_file_name = None

for i, line in enumerate(run_data):
       if f'Run: {hms_run_name}' in line:
              run_found = True
       if 'End:' in line:
              run_found = False
       if run_found and line.strip().startswith('DSS File:'):
              old_dss_file_name = line.split(':')[1].strip()
              run_data[i] = line.replace(old_dss_file_name, hms_run_output_dss)

if old_dss_file_name is None:
       raise Exception("hms_run_name not found in the HMS .run file. Check the run name in the User Input Section")
else:
       print(f"Output DSS file name changed from {old_dss_file_name} to {hms_run_output_dss} in .run file.")

# Write the updated .run data
with open(hms_project_run_file_path, 'w') as file:
       file.writelines(run_data)

# Pause then execute HMS
time.sleep(5)

cmd = [hms_compute_bat_path, os.path.join(hms_project_directory, hms_project_name + '.hms'), hms_run_name]
print(f"Executing HMS with command: {' '.join(cmd)}")

# Execute HMS
subprocess.run(cmd)

# HEC-HMS creates a log file with the hms_run_name with a .log extension
# To avoid overwriting the log file, we rename it to include the run number and the suffix (matching the DSS file name with a .log extension)
logfile_path = os.path.join(hms_project_directory, hms_run_output_dss.replace('.dss', '.log'))

# Remove existing log file, if it exists
if os.path.exists(logfile_path):
       os.remove(logfile_path)

# Now rename the default HMS log file to match DSS file naming convention
os.rename(os.path.join(hms_project_directory, hms_run_name + '.log'), logfile_path) if os.path.exists(os.path.join(hms_project_directory, hms_run_name + '.log')) else print(f"The file {os.path.join(hms_project_directory, hms_run_name + '.log')} does not exist.")
print(f"Renamed log file to {logfile_path}")

       
# Restore original HMS files
shutil.copy(hms_basin_file_backup_path, hms_basin_file_path)
print(f"Restored {hms_basin_file_backup_path} to {hms_basin_file_path}")
print("")

# Delete the lock file for file backup and restoration
if os.path.exists(lock_file_path):
       os.remove(lock_file_path)
       print("Lock file deleted.")
else:
       print("Lock file does not exist.")

print(f"All Runs Completed!!!!!!!!!!")
print('FINISH-----------------')



Output DSS file does not exist. Preparing HMS Run.
Output DSS file name changed from edc_run_v1_HMS_Run_2_baseline.dss to edc_run_v1.dss in .run file.
Executing HMS with command: D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\Python\HMScompute.bat D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\hms_edc_amacan.hms edc_run_v1
The file D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\edc_run_v1.log does not exist.
Renamed log file to D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\edc_run_v1.log
Restored D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX Amacan Water Balance HH\06 WORK FILES\05 HEC HMS\hms_edc_amacan - Copy\EDC_Basin.basin.bak to D:\AMH Philippines, Inc\EDC Projects - 20 NP24.XXX A